In [ ]:
from microscopy.trainers import train_configuration
from hydra import compose, initialize
import os

def load_path(dataset_root, dataset_name, folder):
    if folder is not None:
        return os.path.join(dataset_root, dataset_name, folder)
    else:
        return None

In [ ]:
# Run to specify GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
config_name =  "config" #@param ["EM_crap_2x_03", "EM_crap_2x_05", "EM_crap_2x_09", "EM_crap_4x_03", "EM_crap_4x_05", "EM_crap_4x_09", "EM_down_2x", "EM_down_4x", "EM_old_crap_4x", "wo_level_1"] 

dataset_name = "F-actin_wo_level1" #@param ["LiveFActinDataset", "EM", "F-actin", "ER", "MT", "MT-SMLM_registered"]

model_name = "unet" #@param ["unet", "rcan", "dfcan", "wdsr", "wgan", "esrganplus", "cddpm"] 
batch_size = 4 #@param {type:"integer"}
num_epochs =  100 #@param {type:"integer"}
lr = 0.001 #@param {type:"number"}
discriminator_lr = 0.001 #@param {type:"number"}

scheduler = 'OneCycle' #@param {'Fixed', 'ReduceOnPlateau', 'OneCycle', 'CosineDecay', 'MultiStepScheduler'}
discriminator_scheduler = 'OneCycle' #@param {'Fixed', 'ReduceOnPlateau', 'OneCycle', 'CosineDecay', 'MultiStepScheduler'}
optimizer = 'adam'  #@param {'adam', 'adamW', 'adamax', 'rms_prop', 'sgd'}
discriminator_optimizer = 'adam'  #@param {'adam', 'adamW', 'adamax', 'rms_prop', 'sgd'}
    
saving_folder = "" #@param {type:"string"}
vebose = True #@param {type:"boolean"}
on_memory = True #@param {type:"boolean"}

In [ ]:
initialize(version_base=None, config_path="conf", job_name="test_app")
cfg = compose(config_name=config_name)
        
cfg.dataset_name = dataset_name

train_lr, train_hr, val_lr, val_hr, test_lr, test_hr = cfg.used_dataset.data_paths

dataset_root = "datasets" if os.path.exists("datasets") else "../datasets"
train_lr_path = load_path(dataset_root, dataset_name, train_lr)
train_hr_path = load_path(dataset_root, dataset_name, train_hr)
val_lr_path = load_path(dataset_root, dataset_name, val_lr)
val_hr_path = load_path(dataset_root, dataset_name, val_hr)
test_lr_path = load_path(dataset_root, dataset_name, test_lr)
test_hr_path = load_path(dataset_root, dataset_name, test_hr)

cfg.model_name = model_name
cfg.hyperparam.batch_size = batch_size
cfg.hyperparam.num_epochs = num_epochs
cfg.hyperparam.lr = lr
cfg.hyperparam.discriminator_lr = discriminator_lr

cfg.hyperparam.scheduler = scheduler
cfg.hyperparam.discriminator_lr_scheduler = discriminator_scheduler
cfg.hyperparam.optimizer = optimizer
cfg.hyperparam.discriminator_optimizer = discriminator_optimizer

cfg.model.optim.early_stop.patience = num_epochs

save_folder = "scale" + str(cfg.used_dataset.scale)
if cfg.hyperparam.additional_folder is not None:
    save_folder += "_" + cfg.hyperparam.additional_folder

saving_path = os.path.join(
    saving_folder, 
    cfg.dataset_name,
    cfg.model_name,
    save_folder,
    f"epc{cfg.hyperparam.num_epochs}_btch{cfg.hyperparam.batch_size}_lr{cfg.hyperparam.lr}_optim-{cfg.hyperparam.optimizer}_lrsched-{cfg.hyperparam.scheduler}_seed{cfg.hyperparam.seed}_1"
)

test_metric_path = os.path.join(saving_path, "test_metrics")
if (os.path.exists(test_metric_path) and len(os.listdir(test_metric_path)) > 0):
    # In case you want to skip existing configurations
    # continue
    # In case you want to repeat existing configurations
    saving_path = saving_path[:-1] + str(int(saving_path[-1]) + 1) 

model = train_configuration(
    config=cfg,
    train_lr_path=train_lr_path,
    train_hr_path=train_hr_path,
    val_lr_path=val_lr_path,
    val_hr_path=val_hr_path,
    test_lr_path=test_lr_path,
    test_hr_path=test_hr_path,
    saving_path=saving_path,
    verbose=0, # 0, 1 or 2
    data_on_memory=0
)
